In [105]:
from pprint import pprint
import json

with open('bias-study-export.json') as f:
    data = json.load(f)['logs']
    
with open('images/_airlines.json') as f:
    airlines = json.load(f)

with open('images/_states.json') as f:
    states = json.load(f)

pprint(data['82534293'])

{'approximateVis_airline': {'HowMany': {'answer': '4000',
                                        'confidence': 'agree',
                                        'type': 'HowMany'},
                            'HowManyCompare_EV': {'answer': '2000',
                                                  'confidence': 'weak_agree',
                                                  'data': 'EV',
                                                  'type': 'HowManyCompare'},
                            'HowManyCompare_VX': {'answer': '3500',
                                                  'confidence': 'neutral',
                                                  'data': 'VX',
                                                  'type': 'HowManyCompare'}},
 'approximateVis_states': {'HowMany': {'answer': '6000',
                                       'confidence': 'strong_agree',
                                       'type': 'HowMany'},
                           'HowManyCompare_MA': {'answer': '30

In [103]:
from collections import OrderedDict 
from copy import deepcopy

rows = []

for p in data.items():
    d = OrderedDict({
        'id': p[0]
    })
    
    item = p[1]
    
    d['userAgent'] = item['userAgent']
    d['isMobile'] = item['isMobile']
    d['done'] = 'done' in item
    
    cond = item['cond']
    
    errorAirline = cond['errorAirline']
    seqAirline = cond['seqAirline']
    errorStates = cond['errorStates']
    seqStates = cond['seqStates']
    focusAirline = cond['focusAirline']
    focusState = cond['focusState']
    
    for k, v in cond.items():
        d[k] = v
        
    howManyIndex = 0
    for k, v in item['approximateVis_airline'].items():
        t = v['type']
        
        truth = airlines["{}_{}".format(errorAirline, seqAirline)]
        
        prefix = 'approx.airline.'
        
        if (t == 'HowMany'):
            p = prefix + 'howMany.'
            d[p + 'answer'] = int(v['answer'])
            d[p + 'true'] = truth[focusAirline]
            d[p + 'confidence'] = v['confidence']
            
        if (t == 'HowManyCompare'):
            p = prefix + 'howManyCompare.' + str(howManyIndex) + '.'
            d[p + 'answer'] = int(v['answer'])
            d[p + 'data'] = v['data']
            d[p + 'true'] = truth[v['data']]
            d[p + 'confidence'] = v['confidence']
            howManyIndex += 1
    
    howManyIndex = 0
    for k, v in item['approximateVis_states'].items():
        t = v['type']
        
        truth = states["{}_{}".format(errorStates, seqStates)]
        
        prefix = 'approx.states.'
        
        if (t == 'HowMany'):
            p = prefix + 'howMany.'
            d[p + 'answer'] = int(v['answer'])
            d[p + 'true'] = truth[focusState]
            d[p + 'confidence'] = v['confidence']
            
        if (t == 'HowManyCompare'):
            p = prefix + 'howManyCompare.' + str(howManyIndex) + '.'
            d[p + 'answer'] = int(v['answer'])
            d[p + 'data'] = v['data']
            d[p + 'true'] = truth[v['data']]
            d[p + 'confidence'] = v['confidence']
            howManyIndex += 1
     
    def jaccard_similarity(x,y):
        intersection_cardinality = len(set.intersection(*[set(x), set(y)]))
        union_cardinality = len(set.union(*[set(x), set(y)]))
        return intersection_cardinality/float(union_cardinality)
    
    # precise
            
    howManyIndex = 0
    for k, v in item['preciseVis_airline'].items():
        t = v['type']
        
        truth = airlines["{}_{}".format(errorAirline, seqAirline)]
        
        prefix = 'precise.airline.'
        
        if (t == 'HowMany'):
            p = prefix + 'howMany.'
            d[p + 'answer'] = int(v['answer'])
            d[p + 'true'] = truth[focusAirline]
            d[p + 'confidence'] = v['confidence']
            
        if (t == 'HowManyCompare'):
            p = prefix + 'howManyCompare.' + str(howManyIndex) + '.'
            d[p + 'answer'] = int(v['answer'])
            d[p + 'data'] = v['data']
            d[p + 'true'] = truth[v['data']]
            d[p + 'confidence'] = v['confidence']
            howManyIndex += 1
            
        if (t == 'DidYouNotice'):
            p = prefix + 'didYouNotice.'
            d[p + 'answer'] = v['answer'] == 'yes'
            d[p + 'true'] = errorAirline > 1
            d[p + 'confidence'] = v['confidence']
            
        if (t == 'SelectAll'):
            v = deepcopy(v)
            p = prefix + 'SelectAll.'
            d[p + 'confidence'] = v['confidence']
            
            del v['confidence']
            del v['type']
            
            answers = v.keys()
            t = [k for k,v in truth.items() if v]
            d[p + 'answer'] = ','.join(answers)
            d[p + 'true'] = ','.join(t)
            d[p + 'jaccard'] = jaccard_similarity(answers, t)
            
    howManyIndex = 0
    for k, v in item['preciseVis_states'].items():
        t = v['type']
        
        truth = states["{}_{}".format(errorStates, seqStates)]
        
        prefix = 'precise.states.'
        
        if (t == 'HowMany'):
            p = prefix + 'howMany.'
            d[p + 'answer'] = int(v['answer'])
            d[p + 'true'] = truth[focusState]
            d[p + 'confidence'] = v['confidence']
            
        if (t == 'HowManyCompare'):
            p = prefix + 'howManyCompare.' + str(howManyIndex) + '.'
            d[p + 'answer'] = int(v['answer'])
            d[p + 'data'] = v['data']
            d[p + 'true'] = truth[v['data']]
            d[p + 'confidence'] = v['confidence']
            howManyIndex += 1
            
        if (t == 'DidYouNotice'):
            p = prefix + 'didYouNotice.'
            d[p + 'answer'] = v['answer'] == 'yes'
            d[p + 'true'] = errorStates > 1
            d[p + 'confidence'] = v['confidence']
            
        if (t == 'SelectAll'):
            v = deepcopy(v)
            p = prefix + 'SelectAll.'
            d[p + 'confidence'] = v['confidence']
            
            del v['confidence']
            del v['type']
            
            answers = v.keys()
            t = [k for k,v in truth.items() if v]
            d[p + 'answer'] = ','.join(answers)
            d[p + 'true'] = ','.join(t)
            d[p + 'jaccard'] = jaccard_similarity(answers, t)
    
    for k, v in item['timing'].items():
         d['timing.' + k] = v
    
    rows.append(d)

pprint(rows)

[OrderedDict([('id', '82534293'),
              ('userAgent',
               'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_1) '
               'AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2883.59 '
               'Safari/537.36'),
              ('isMobile', False),
              ('done', True),
              ('seqAirline', 1),
              ('studyCondition', 'onlyNew'),
              ('secondCondition', 'airline'),
              ('seqStates', 11),
              ('errorStates', 1000),
              ('focusState', 'NY'),
              ('errorAirline', 1),
              ('focusAirline', 'AA'),
              ('firstCondition', 'states'),
              ('approx.airline.howManyCompare.0.answer', 2000),
              ('approx.airline.howManyCompare.0.data', 'EV'),
              ('approx.airline.howManyCompare.0.true', 1998),
              ('approx.airline.howManyCompare.0.confidence', 'weak_agree'),
              ('approx.airline.howManyCompare.1.answer', 3500),
              ('approx

In [104]:
import csv

with open('bias-study-export.csv', 'w') as f:
    fieldnames = rows[0].keys()
    writer = csv.DictWriter(f, fieldnames=fieldnames)
    writer.writeheader()
    
    for row in rows:
        writer.writerow(row)

        
import pandas as pd

pd.read_csv("bias-study-export.csv")

,id,userAgent,isMobile,done,seqAirline,studyCondition,secondCondition,seqStates,errorStates,focusState,...,timing.approximate_first_html,timing.precise_first_html,timing.index_html,timing.approximate_questions_second_html,timing.thanks_html,timing.precise_questions_html,timing.approximate_second_html,timing.instructions_html,timing.demographics_html,timing.precise_instructions_html
0,82534293,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_1...,False,True,1,onlyNew,airline,11,1000,NY,...,40055,55636,8695,29924,1199315,187490,145279,26016,12491,1501
